In [16]:
# Libaries 
# Libraries and Modules used
import MetaTrader5 as mt5 
import pandas as pd
from sklearn.linear_model import LinearRegression
import ta
import warnings
warnings.filterwarnings("ignore")
import numpy as np
# warnings.filterwarnings("ignore")
mt5.initialize()
# Replace following with your MT5 Account Login
account=51127988 # 
password="Aar2frM7"
server = 'ICMarkets-Demo'

# Function to get rates from MT5 
def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1

# GARCH / ARCH 
Steps are:
1. Specify the model
2. Fit the model
3. Make a forecast

Model Assumptions:
- Distributions: normal, "t" (T - Distribution), "skewt" (Skewed t-distributions)
- Mean model: constant, "zero", "AR"
- Volatility Model: GARCH / ARCH 



In [20]:
from arch import arch_model

In [21]:
EURUSD = get_rates('EURUSD.a', mt5.TIMEFRAME_H4, 2000,)

In [41]:
returns = EURUSD['close'].pct_change().dropna() * 100

In [42]:
returns

1      -0.006716
2       0.106498
3       0.038337
4      -1.252180
5      -0.146501
          ...   
1995    0.104374
1996    0.036019
1997   -0.153499
1998   -0.271409
1999   -0.000952
Name: close, Length: 1999, dtype: float64

In [44]:
basic_gm = arch_model(returns, p = 1, q = 1, mean = 'constant', vol = 'GARCH', dist = 'normal')
gm_result = basic_gm.fit(update_freq = 4)
print(gm_result.params)
print(gm_result.summary())

Iteration:      4,   Func. Count:     33,   Neg. LLF: 360.09377666374985
Iteration:      8,   Func. Count:     61,   Neg. LLF: 247.27141196134394
Iteration:     12,   Func. Count:     93,   Neg. LLF: 1262.2528529756364
Iteration:     16,   Func. Count:    119,   Neg. LLF: 7306916.357012011
Optimization terminated successfully    (Exit mode 0)
            Current function value: -64.30165944128586
            Iterations: 23
            Function evaluations: 143
            Gradient evaluations: 19
mu         -0.001440
omega       0.000385
alpha[1]    0.012597
beta[1]     0.980071
Name: params, dtype: float64
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                  close   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                64.3017
Distribution:                  Normal   AIC:                         

In [53]:
# Making Predictions

gm_forecast = gm_result.forecast(horizon = 5)
print(gm_forecast.varianceiance)

           h.1      h.2       h.3       h.4       h.5
1999  0.042487  0.04256  0.042633  0.042705  0.042777
